# Amazon SageMaker JumpStart で作成した PyTorch モデルを使ってバッチ推論を行う

In [ ]:
%matplotlib inline
import boto3
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from shutil import copyfile
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()
boto_session = boto3.session.Session()
region = boto_session.region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-jumpstart-batch-inference-script"
print("Bucket:\n{}".format(bucket))

以下のセルの `inference_inputs` にバッチ推論したい画像が保存されている S3 パスを、`model_path` に JumpStart の学習済みモデルの S3 パスを設定してから実行してください。

In [ ]:
inference_inputs = 's3://bucket/prefix'
model_path = 's3://bucket/job_name/output/model.tar.gz'
model_dir = model_path[:-12]

バッチ推論で使用するコンテナの仕様に合わせるため、学習済みモデルの名前をリネームして再度 tar.gz で固めて S3 に保存します。

In [ ]:
!aws s3 cp $model_path ./
!tar zxvf model.tar.gz
!mv model.pt model.pth
!tar zcvf model2.tar.gz model.pth class_label_to_prediction_index.json
!aws s3 cp model2.tar.gz $model_dir

バッチ推論用のコード `inference.py` を使って `PyTorchModel` を作成し、バッチ変換ジョブを実行します。

In [ ]:
model = PyTorchModel(
            entry_point='inference.py',
            model_data = model_path,
            role=role,
            py_version="py3",
            framework_version='1.8.1')

In [ ]:
# output_path = 's3://xxx'  # 結果出力パスを指定する場合はこちらを使用
transformer = model.transformer(
                instance_count=1,
                instance_type='ml.c5.xlarge',
#                 output_path=output_path
                )

In [ ]:
transformer.transform(
    data=inference_inputs,
    data_type="S3Prefix",
    content_type="application/x-image",
    wait=True,
)